# Basics of image processing with python

## Lesson 3

In this lesson, we will investigate color properties and possibilities to manipulate color information.

### 3.1 Color spaces

Images are typically captured and dsaved in the RGB-color space. Also, if you read in images with `cv2.imread`, this model is applied (however, the channel order is reersed from RGB to BGR). Let us assume you have read in a color image `myImg` and would like to convert it to another colorspace. The ch2 library offers several possibilities for color transformation with the `cv2.cvtColor` method. The first argument corresponds to the image you would like to transform, the second the type of transformation you would like to apply.

First let's see how to convert an RGB (BGR)-image to the HSV-color space. Here, you use the `cv2.COLOR_BGR2HSV` transformation. The result will be a 3-dimensional array, with the first two dimensions corresponding to the spatial axes and the third dimension corresponding to the hue, saturation and value channel. You can then use these three channels and manipulate them individually.

By the way, other transformations are also possible. For example, `cv2.COLOR_BGR2GRAY` will transform a color image to grayscale. For more information, please check the documenttaion.

In [ ]:
myImg_hsv = cv2.cvtColor(myImg, cv2.COLOR_BGR2HSV)

myImg_hue = myImg_hsv[:, :, 0]
myImg_saturation = myImg_hsv[:, :, 1]
myImg_value = myImg_hsv[:, :, 2]

You can now manipulate the three parameters individually, for example, double the saturation. Be aware of the data type you are using: if you e.g. use an 8-bit image and your pixel has a saturation of 180, you will see a value overflow and doubling the saturation will result in a saturation of 105 if the input is simply multiplied by 2. To prevent this, first convert the saturation to a larger data type, e.g. `np.uint16` and perform the multiplication on that result . Then, clip the result to the allowed value range of your input format, e.g. to 0 to 255 for 8-bit images by using `np.clip()` . Finally, convert the result back to the initial data type with `.astype()`. So, doubling the saturation could e.g. look like this:

In [ ]:
myImg_saturation_doubled = np.clip((myImg_saturation.astype(np.uint16))*2, 0, 255).astype(np.uint8)

To now merge the maniulated hue, saturation and value components back to one image, you can use `cv2.merge`, passing the h, s and v components as arguments as a list. To convert the result back to the RGB(BGR) color space, use `cv2.cvtColor` again, this time with the reverse transformation `cv2.COLOR_HSV2BGR`.

In [ ]:
myImg_hsv_saturationDoubled = cv2.merge([myImg_hue, myImg_saturation_doubled, myImg_value])
myImg_BGR_saturationDoubled = cv2.cvtColor(myImg_hsv_saturationDoubled, cv2.COLOR_BGR2HSV)

### 3.2 White balancing

Differnet possibilities exist to perform white balancing. The method shown here is meant as an example implementation to demonstarte the principle, but does not claim to be a one-size-fits-all solution for all problems.

First, we need to identify a white (or grey) reference region in our image. Let us assume we are performing white balaning on a color image named `myImg`. For this example, we asume that the region is found between pixel 100 and 300 for both coordinates. We then calculate the mean values of the three channels inside this region:

In [ ]:
b_avg = np.mean(myImg[100:300, 100:300, 0])
g_avg = np.mean(myImg[100:300, 100:300, 1])
r_avg = np.mean(myImg[100:300, 100:300, 2])

We can then compute the correction factors for blue and red (`b_corr` and `r_corr`, respectively) by calculating the ratio of the average green intensity and the average blue/red intensity:

In [ ]:
b_corr = g_avg/b_avg
r_corr = g_avg/r_avg

We then apply the correction factors to the respecive channels to obtain a new, white balanced image `myImg_wb`. We can simply copy the existing `myImg` and replace the red and blue channel with the corrected values, whereas the green channel is unchanged.

myImg_wb = myImg
myImg_wb[:, :, 0] = imyImg[:, :, 0]*b_corr
myImg_wb[:, :, 2] = myImg[:, :, 2]*r_corr